# Support Vector Machines (SVM)
SVM is a supervised learning algorithm used for classification. It can be used to perform binary or multiclass classification. SVM works by finding the best line (or hyperplane in high-dimensional data) that separates the given groups. The objective of SVM learning is to find the best parameters so that the hyperplane separates the data and maximizes the margin (For a simple discussion about the equations of SVM, have a look at [this paper](https://cgp.iiarjournals.org/content/cgp/15/1/41.full.pdf)).

The closest points are called support vectors, and they are the points used by SVM to find the best hyperplane.

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/SVM_hyperplane.png" width=600 height=200 />

Advantages of SVM include:
- SVM is effective in high dimensional spaces
- Uses a subset of data points to find the best solution (memory efficient)
- Versatility, as it uses different <b>kernel functions</b> to find the best solutions.


### Kernel Functions
Kernel functions are what makes SVM stand out among other similar machine learning algorithms. Originally, SVM was proposed to be a linear classification method. But what would you do if the data are not linearly separable (like in the photo below)?

In this case, a function (or a transformation if you want) is applied to the data, and then SVM will use the transformed data for its learning.

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/svm_intro.png" width=900 height=300 />

The kernel function in the above image is a custom one, it is applied to to variables, and its function is $f(x_1,x_2) = {x_1}^2+{x_2}^2$

But there are custom functions that you can already use using popular packages like scikit-learn. These include the radial basis function (RBF), polynomial function and linear function. The different kernels have differently-shaped decision boundaries.

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_iris_svc_001.png" width=600 height=500 />

Image reference: [scikit-learn website](https://scikit-learn.org/stable/modules/svm.html#svm-classification)

### Hyperparameters
##### The Hyperparameter C
This is a regularization parameter. It determines the width of the margin. Higher C values give smaller margins, and might be prone to overfitting. While lower C values are associated with larger margins, and the possibility of underfitting. 

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/SVM_hyperplane_2.png" width=600 height=250 />

##### Hyperparameter gamma
This hyperparameter only applies for the RBF kernel function. It determines how tight the decision boundary is. With lower values being associated with wider decision boundary and more points being grouped together (with the risk of low accuracy). Higher values are associated with points closer together being considered similar, and thus smaller boundaries (with the risk of overfitting).

## About the Data Used
This dataset was originally publicly available from the paper [Machine Learning Repository of University of California Irvine](http://archive.ics.uci.edu/). This dataset [consists of characteristics, or features, of cell nuclei taken from breast masses which were sampled using fine-needle aspiration (FNA), a common diagnostic procedure in oncology. The clinical samples used to form this dataset were collected from January 1989 to November 1991.] (Sidey-Gibbons and Sidey-Gibbons 2019) Each sample is then classified as malignant or benign in the 'class' column (1= malignant and 0 = benign).


---
## Data Analysis Steps:
### 1. Import libraries and define functions

These are the functions that you will need for your following analyses. <b>This line calls the libraries and functions written in the utils.py file</b>. I kept these functions in another file just for simplicity. You can access it from [THIS LINK](https://github.com/abdulrahman1123/analysis_examples/blob/main/utils.py) if you need.

In [ ]:
from utils import *
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics

### 
### 2. Load the data



In [ ]:
data_dir = 'https://github.com/abdulrahman1123/analysis_examples/raw/main/breast_cancer_wisconsin.csv'
data = pd.read_csv(data_dir)
data = data.iloc[0:200,:]

data

### 
### 3. Create training and test datasets

The following generally applies for all machine learning algorithms:

The first step to do is to <b>split your data into training and testing datasets</b>. The training dataset will be used to train the model and evaluate parameters, while the testing dataset will only be used at the end to test how well did the model learn.

The training dataset can be further divided into training and validation datasets, where the validation dataset is used to evaluate the parameters in order to reach to the best model. The way we will be doing this is represented in the following figure from [scikit-learn website](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width=440 height=305 />

After splitting the data, we will impute the missing values using the mean of each column. This will be done separately for training and testing datasets.

Then, the data will be normalized (mean = 0 and sd = 1) so that the model can coverge faster (i.e. reach the best solution faster)

In [ ]:
# Choose features
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]

# Divide into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

print(f"Size of training dataset = {X_train.shape} and testing dataset = {X_test.shape}")

# Using the mean, impute the training and testing datasets separately
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp.fit_transform(X_train.copy())
X_test = imp.transform(X_test.copy())


# scale both X_train and X_test for faster convergence
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

plot_train_test(X_train,X_test)

### 
###  4. Create the model and find the best hyperparameters
##### What is the best kernel type? What is the best C value and what gamma value should I use?

In order to find the best hyperparameters, we can use grid search cross validation. Using this method, a grid of all possible parameters is checked, and the set of parameters associated with the best model is then chosen. Luckily, this is easily implemented using the GridSearchCV from scikit learn, all we need to do is determine the grid of parameters to be used.

First, lets create the appropriate grid of hyperparameters.

In [ ]:
param_grid = [
  {'C': np.logspace(-2,6, 10), 'kernel': ['linear','poly']},
  {'C': np.logspace(-2,6, 10), 'gamma': np.logspace(-6, 3, 10), 'kernel': ['rbf']},
 ]

grid_model = GridSearchCV(SVC(), param_grid, scoring="accuracy", cv=StratifiedKFold(n_splits=5))

print ("\nSearching for the model with best parameters ...\n")

# fit the model to the training data set
grid_model.fit(X_train, y_train)

print('Best score achieved during training = ',np.round(grid_model.best_score_,2))

print('\nThis is the model with best parameters found')
grid_model.best_estimator_

### 
### 5. Apply the model on the testing data set

In [ ]:
y_pred = grid_model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sens = tp / (tp + fn)
spec = tn / (tn + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)
pd.DataFrame([[sens, spec, accuracy]], columns=['Sensitivity', 'Specificity', 'Accuracy'])

### 
### 5.1. Plot ROC Curve and Print AUC

In [ ]:
best_model = grid_model.best_estimator_
best_model.probability = True
best_model.fit(X_train,y_train)

y_probas =grid_model.predict_proba(X_test)[::,1]

_, ax = plt.subplots(figsize = (6.5,5))
metrics.RocCurveDisplay.from_predictions(y_test, y_probas,name="Malignant vs Benign", color="darkred", plot_chance_level=True,ax=ax)
plt.xlabel("False Positive Rate", font = "Calibri", fontsize = 16)
plt.ylabel("True Positive Rate", font = "Calibri", fontsize = 16)
ax.set_xticks(labels=['0%', '20%', '40%', '60%', '80%', '100%'], ticks=np.arange(0,1.1,0.2), font = "Calibri", fontsize = 12)
ax.set_yticks(labels=['0%', '20%', '40%', '60%', '80%', '100%'], ticks=np.arange(0,1.1,0.2), font = "Calibri", fontsize = 12)
plt.title("Reciever Operating Characterstic for Predicting\nCancer Malignancy", font = "Calibri", fontsize = 18,weight = 'bold')
plt.legend()

# 
---
# References and Further Reading
[Introduction to Support Vector Machines (SVM)](https://www.geeksforgeeks.org/introduction-to-support-vector-machines-svm/)

[Classifying data using Support Vector Machines(SVMs) in Python](https://www.geeksforgeeks.org/classifying-data-using-support-vector-machinessvms-in-python/)

[Huang, Shujun, et al. "Applications of support vector machine (SVM) learning in cancer genomics." Cancer genomics & proteomics 15.1 (2018): 41-51.‏](https://cgp.iiarjournals.org/content/cgp/15/1/41.full.pdf))

[Scikit-Learn Website: Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html)

[Scikit-learn website: Grid Search](https://scikit-learn.org/stable/modules/grid_search.html#grid-search)